In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import random
import re
import os

In [2]:
data_file=os.listdir('./CG_graph')

In [ ]:
for filename in data_file:
    if filename.endswith('.txt'):
        print(filename)

        data=pd.read_csv('./final_file/'+filename,sep='\t',header=None)
        n=len(data)

        g = nx.Graph()
        for (u,v) in data.values:
            g.add_edge(u,v)
        degree_dict=dict(g.degree)
        degree=list(set(degree_dict.values()))
        degree.sort()
        print('the length of degree is {}'.format(len(degree)))
        degree_size=int(len(degree)/16)
        degree_one=degree[degree_size*2]
        degree_two=degree[degree_size*4]
        degree_three=degree[degree_size*6]
        degree_four=degree[degree_size*8]

        node_low=[k for k,v in degree_dict.items() if v < degree_one]
        node_middle=[k for k,v in degree_dict.items() if v >= degree_two and v < degree_three]
        node_high=[k for k,v in degree_dict.items() if v >= degree_four]
        print(len(node_low),len(node_middle),len(node_high))
        print(n)
        for batch_size in [500,1000,2000,5000]:
            print(batch_size)
            for way in ['add','minus']:
                if way=='add':
                    print(way)
                    situation_dic={'low':node_low,'middle':node_middle,'high':node_high}
                    for degree_situation in ['low','middle','high']:
                        node_=situation_dic[degree_situation]
                        node_choise = list(np.random.choice(node_, size=batch_size, replace=True))
                        data_final=pd.DataFrame()
                        sign=['+']*(n+batch_size)
                        in_=list(data[0].values)
                        out_=list(data[1].values)

                        for u in node_choise:
                            v=random.sample(g.nodes(),1)[0]
                            while g.has_edge(u,v) and u==v:
                                v=random.sample(g.nodes(),1)[0]
                            g.add_edge(u,v)
                            in_.append(u)
                            out_.append(v)

                        data_final['sign']=sign
                        data_final['in']=in_
                        data_final['out']=out_
                        name=filename.strip('.txt')
                        data_final.to_csv('./degree/add/'+name+'_'+'batchsize_'+str(batch_size)+'_'+degree_situation+'_add_degree.txt',sep='\t',index=None,header=None)
                else:
                    print(way)
                    situation_dic={'low':node_low,'middle':node_middle,'high':node_high}
                    for degree_situation in ['low','middle','high']:
                        node_=set(situation_dic[degree_situation])
                        #random.sample(node_,1)[0]
                        #node_choise=random.sample(node_,batch_size)
                        i=0
                        data_final=pd.DataFrame()
                        sign=['+']*n
                        sign.extend(['-']*batch_size)
                        in_=list(data[0].values)
                        out_=list(data[1].values)
                        while i<batch_size:
                            u,v=random.sample(g.edges(),1)[0]
                            if u in node_ or v in node_:
                                g.remove_edge(u,v)
                                in_.append(u)
                                out_.append(v)
                                i=i+1

                        data_final['sign']=sign
                        data_final['in']=in_
                        data_final['out']=out_
                        name=filename.strip('.txt')
                        data_final.to_csv('./degree/minus/'+name+'_'+'batchsize_'+str(batch_size)+'_'+degree_situation+'_minus_degree.txt',sep='\t',index=None,header=None)

SSCA_17.txt
the length of degree is 55
1185 9985 101265
2218744
500
add
minus


In [ ]:
data=pd.read_csv('./PLG_graph/'+file_name,sep='\t',header=None)

In [26]:
import os

file=os.listdir('../max_clique/clean')
print(file)

['PLG_5000000_4991006_add_minus.txt', 'soc-livejournal_add_minus.txt', 'Email-EuAll_add_minus.txt', 'wikipedia-growth_add_minus.txt', 'ca-cit-HepTh_add_minus.txt', 'SSCA_17_add_minus.txt', '.ipynb_checkpoints', 'ER_1000000_4999725_add_minus.txt', 'SSCA_13_add_minus.txt', 'ER_100000_2502270_add_minus.txt', 'dblp_coauthor_add_minus.txt', 'flickr-growth_add_minus.txt', 'PLG_500000_499373_add_minus.txt']


In [91]:
file=['Email-EuAll.txt']
for file_name in file:
    if file_name.endswith('.txt'):
        data=pd.read_csv(file_name,sep='\t',header=None)
        a=['+']*len(data)
        a.extend(['-']*len(data))
        a=np.array(a)
        data_end=pd.DataFrame()
        data_end[0]=a
        in_=data[0].values
        out_=data[1].values
        data_end[1]=np.hstack((in_,in_[::-1]))
        data_end[2]=np.hstack((out_,out_[::-1]))
        name=file_name.strip('.txt')
        print(name)
        data_end.to_csv('./'+name+'_add_minus.txt',sep='\t',index=None,header=None)

Email-EuAll


       0  1
0      0  1
1    175  3
2    457  3
3  75912  3
4      2  3
5      0  4
6      4  5
7      0  5
8      6  7
9  74996  7


In [89]:
filename='./real_data/Email-EuAll.txt'

G = nx.DiGraph()
i=0

with open(filename) as file:
    for line in file:
        if i<4:
            i+=1
            continue
        head, tail= [int(x) for x in re.split('[ ,\t]',line)]  #如果节点使用数组表示的可以将str(x)改为int(x)
        if head!=tail:
            if head>tail:
                head,tail=tail,head
            G.add_edge(head,tail)

DVweight = G.degree()
max_degree = max(span for n, span in DVweight)		#节点最大度数


ver_num=len(G.nodes())
edge_num=len(G.edges())
print(filename,ver_num,edge_num,(2*edge_num)/(ver_num*ver_num),max_degree)
data_list=[[edge[0],edge[1]] for edge in G.edges()]
data=pd.DataFrame(data=data_list,columns=['in','out'])
print(len(data))
name=filename.split('/')[-1]
data.to_csv('./'+name,sep='\t',index=None,header=None)

./real_data/Email-EuAll.txt 265009 364481 1.0379672312172553e-05 7636
364481


In [93]:
filename='Email-EuAll.txt'
data=pd.read_csv(filename,sep='\t',header=None)
data=data.sort_values(axis=0, ascending=True, by=1).reset_index(drop=True)
print(data.head(10))
data.to_csv('./Email-EuAll.txt',sep='\t',index=None,header=None)

   0   1
0  0   1
1  2   3
2  0   4
3  4   5
4  0   5
5  6   7
6  0   8
7  3  10
8  5  10
9  9  10
